In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [ ]:
import json
with open('natural_question.json') as f: 
    data = json.load(f)
len(data)

5249

In [ ]:
question_data = []
count = 0 
question_output = {}
for i in range(len(data)): 
    if data[str(i+1)][0]['question_template_id'] == '4q2': 
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if "confirmed cases" in ent[3] and "sex" in ent[0]:
            table = 'case_sex.csv'
        elif "deaths" in ent[3] and "sex" in ent[0]: 
            table = 'death_sex.csv'
        elif "confirmed cases" in ent[3] and "age" in ent[0]: 
            table = "case_age.csv"
        elif "deaths" in ent[3]  and "age" in ent[0]:
            table = "death_age.csv"
        elif "confirmed cases" in ent[3] and ("race" in ent[0] or "ethnicity" in ent[0]): 
            table = "case_race.csv"
        elif "deaths" in ent[3] and ("race" in ent[0] or "ethnicity" in ent[0]): 
            table = "death_race.csv"
        else: 
            continue
        if "percentage" in q: 
            col = "perc"
        else:
            col = "num"
            
        print(count)
        if table == "case_sex.csv" and col == "num": 
            df = pd.read_csv('case_sex.csv', skiprows=2)
            df = df[['Sex', 'Count of cases']]
            df = df.rename(columns = {"Sex" : "sex", "Count of cases" : "cases"})
            df = df.astype(str)
        elif table == "case_sex.csv" and col == "perc": 
            df= pd.read_csv("case_sex.csv", skiprows = 2)
            df = df[['Sex', 'Percent of cases']]
            df = df.rename(columns = {"Sex" : "sex", "Percent of cases" : "cases"})
            df = df.astype(str)
        elif table == "death_sex.csv" and col == "num": 
            df = pd.read_csv('death_sex.csv', skiprows=2)
            df = df[['Sex', 'Count of deaths']]
            df = df.rename(columns = {"Sex" : "sex", "Count of deaths" : "deaths"})
            df = df.astype(str)
        elif table == "death_sex.csv" and col == "perc": 
            df = pd.read_csv('death_sex.csv', skiprows=2)
            df = df[['Sex', 'Percentage of deaths']]
            df = df.rename(columns = {"Sex" : "sex", "Percentage of deaths" : "deaths"})
            df = df.astype(str)
        elif table == "case_age.csv" and col == "num":
            df = pd.read_csv('case_age.csv', skiprows=2)
            df = df[['Age Group', 'Count of cases']]
            df = df.rename(columns = {"Age Group" : "age", "Count of cases" : "cases"})
            df = df.astype(str)
        elif table == 'case_age.csv' and col == 'perc': 
            df = pd.read_csv('case_age.csv', skiprows=2)
            df = df[['Age Group', 'Percent of cases']]
            df = df.rename(columns = {"Age Group" : "age", "Percent of cases" : "cases"})
            df = df.astype(str)
        elif table == 'death_age.csv' and col == 'num': 
            df = pd.read_csv('death_age.csv', skiprows=2)
            df = df[['Age Group', 'Count of deaths']]
            df = df.rename(columns = {"Age Group" : "age", "Count of deaths" : "deaths"})
            df = df.astype(str)
        elif table == 'death_age.csv' and col == 'perc': 
            df = pd.read_csv('death_age.csv', skiprows=2)
            df = df[['Age Group', 'Percentage of deaths']]
            df = df.rename(columns = {"Age Group" : "age", "Percentage of deaths" : "deaths"})
            df = df.astype(str)
        elif table == 'case_race.csv' and col == 'num':
            df = pd.read_csv('case_race.csv', skiprows=2)
            df = df[['Race/Ethnicity', 'Count of cases']]
            df = df.rename(columns = {"Count of cases": "cases"})
            df = df.astype(str)
        elif table == 'case_race.csv' and col == 'perc':
            df = pd.read_csv('case_race.csv', skiprows = 2)
            df = df[['Race/Ethnicity', 'Percent of cases']]
            df = df.rename(columns = {"Percent of cases" : 'cases'})
            df = df.astype(str)
        elif table == 'death_race.csv' and col == 'num':
            df = pd.read_csv('death_race.csv', skiprows=2)
            df = df[['Race/Ethnicity', 'Count of deaths']]
            df = df.rename(columns = {"Count of deaths": "deaths"})
            df = df.astype(str)
        else: 
            df = pd.read_csv('death_race.csv', skiprows = 2)
            df = df[['Race/Ethnicity', 'Percent of deaths']]
            df = df.rename(columns = {"Percent of deaths" : 'deaths'})
            df = df.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df.columns)
        list_of_list.extend(df.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        res = ""
        count1 = 0
        for i in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[i[0]+1][i[1]]
            else: 
                res += ', ' + list_of_list[i[0]+1][i[1]]
            question_output[q] = res
        count += 1

question_output

0
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


sex,cases
Female,13293121
Male,12171907
Other,490



> Give me the sex with the highest number of confirmed cases in the United States.
Female
1
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


sex,cases
Female,13293121
Male,12171907
Other,490



> Give me the sex with the lowest number of confirmed cases in the United States.
Other
2
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...

Race/Ethnicity,deaths
Hispanic/Latino,18.8
American Indian / Alaska Native Non-Hispanic,0.9
Asian Non-Hispanic,4.0
Black Non-Hispanic,13.6
Native Hawaiian / Other Pacific Islander Non-Hispanic,0.2
White Non-Hispanic,58.6
Multiple/Other Non-Hispanic,3.9



> Give me the ethnicity with the most percentage of deaths in the United States.
Hispanic/Latino
95
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Race/Ethnicity,deaths
Hispanic/Latino,18.8
American Indian / Alaska Native Non-Hispanic,0.9
Asian Non-Hispanic,4.0
Black Non-Hispanic,13.6
Native Hawaiian / Other Pacific Islander Non-Hispanic,0.2
White Non-Hispanic,58.6
Multiple/Other Non-Hispanic,3.9



> Give me the ethnicity with the least percentage of deaths in the United States.
American Indian / Alaska Native Non-Hispanic


{'Give me the sex with the highest number of confirmed cases in the United States.': 'Female',
 'Give me the sex with the lowest number of confirmed cases in the United States.': 'Other',
 'Give me the sex with the most number of confirmed cases in the United States.': 'Female',
 'Give me the sex with the least number of confirmed cases in the United States.': 'Other',
 'Give me the sex with the highest percentage of confirmed cases in the United States.': 'Female',
 'Give me the sex with the lowest percentage of confirmed cases in the United States.': 'Other',
 'Give me the sex with the most percentage of confirmed cases in the United States.': 'Female',
 'Give me the sex with the least percentage of confirmed cases in the United States.': 'Other',
 'Give me the sex with the lowest number of deaths in the United States.': 'Other',
 'Give me the sex with the least number of deaths in the United States.': 'Other',
 'Give me the sex with the lowest percentage of deaths in the United Stat

In [ ]:
with open('4q2data.json', 'w') as outfile:
    json.dump(question_output, outfile)

In [ ]:
len(question_output)

82